In [1]:
"""###################################################################
Module to be loaded before running the script:
- ANTs/git
- qbatch/git

###################################################################"""
print "This box contains notes for myself"

This box contains notes for myself


In [2]:
# ====================================================================
# IMPORT LIBRARIES
# ====================================================================
import os
import subprocess
import sys
import glob #for wildcard matching
import time



In [3]:
# ====================================================================
# Setup directories, paths and filenames
# - All paths must be ABSOLUTE!
# - Refer to 'glob' manual to see the wildcard documentation for file-
#        name matching
# 
# ====================================================================
""" Note on directory hierarchy for the input, reference and transform files 
parent_dir
    |
    +-- subj_dir_1 (as indicated by dir_wc)
    |        |
    |        +- file_of_interest (as indicated by file_wc)
    |
    +-- subj_dir_2 ...
"""

# =====================================
# Input related
# =====================================
######## Input file paths ########
# Path to the parent directory of the input files
in_parent_dir='/data/chamal/projects/anthony/nmf_parcellation/cortical_tractMap/tract2voxel_probability_labels/native_space'
# Wildcard / name of the individual subject directories
in_dir_wc='[0-9][0-9][0-9][0-9][0-9][0-9]'
# Wildcard / name of the input file
in_file_wc='*.nii.gz'

######## Reference / template file paths ########
ref_parent_dir='/data/chamal/projects/anthony/HCP/raw_files/anthony'
ref_dir_wc='[0-9][0-9][0-9][0-9][0-9][0-9]'
ref_file_wc='HCP100unrelated-fa_template0_x07.nii.gz'

######## Transform file paths ########
tr_parent_dir='/data/chamal/projects/anthony/HCP/raw_files/anthony'
tr_dir_wc='[0-9][0-9][0-9][0-9][0-9][0-9]'
tr_file_wc='HCP100unrelated-fa_*_dt_fa*'


# =====================================
# Output related
# =====================================
######## Output file paths ########
out_parent_dir='/data/chamal/projects/anthony/nmf_parcellation/cortical_tractMap/tract2voxel_probability_labels/model_space'
 # NOTE:  output directory will be made by the subject ID
def generate_output_file_name(input_file_name): #Function to make output file name
    name_parts = input_file_name.split('.')
    name_parts[0] += '_modelSpace'
    returnable = '.'.join(name_parts)
    returnable = returnable.replace('_comp3_','_comp5') #TEMP FIX DELETE AFTER THIS RUN
    return returnable 

######## Job document paths ########
jobDoc_dir='/data/chamal/projects/anthony/qbatch_jobDocs'
jobDoc_name='seg5_voxelizedTracts_native2Model_jobDoc.sh'
jobList_name='autoSubmit_seg5_voxelizedTracts_native2Model_jobList.sub'


# =====================================
# antsApplyTransform related
# =====================================
DIMENSIONALITY='3'
INTERPOLATION='GenericLabel'


In [4]:
# ====================================================================
# Save the filenames into lists
# ====================================================================

#### Initilize and save input files
in_file_list=glob.glob(os.path.join(in_parent_dir,in_dir_wc,in_file_wc))
in_file_list.sort()

#### Initialize and save reference files
ref_file_list=glob.glob(os.path.join(ref_parent_dir,ref_dir_wc,ref_file_wc))
ref_file_list.sort()

#### Initialize and save transform files
tr_file_list=glob.glob(os.path.join(tr_parent_dir,tr_dir_wc,tr_file_wc))
tr_file_list.sort()

#Printout
print "File initilization stats:"
print "\tTotal input files: %d" % len(in_file_list)
print "\tTotal reference files: %d" % len(ref_file_list)
print "\tTotal transform files: %d" % len(tr_file_list)


File initilization stats:
	Total input files: 1467
	Total reference files: 163
	Total transform files: 200


In [5]:
# ====================================================================
# Merge paths in lists into dictionaries for each subjet
# - One does not need to use the above methods to get to this step;
#       the below processing steps will work as long as this dict
#       is initialized with the correct information
# - Currently, the merging is done via string-matching subject ID
#       in the file names
# ====================================================================

# Get a list of subject IDs
subject_list_path = "/data/chamal/projects/anthony/nmf_parcellation/cortical_tractMap/subj_list.txt" 
IDs=[]
f = open(subject_list_path, "r")
for subj in f:
    IDs.append(subj.strip())
f.close()

#Initilize the master dictionary to store subject filepaths
unordered_subj_files = {}
#Initialize all the sub-data structures within the dictionary
for ID in IDs:
    unordered_subj_files[ID] = {} #Initialize the individual subj dict
    unordered_subj_files[ID]['in']=[] #Initialize list for input files
    unordered_subj_files[ID]['tr']=[] #Initialize list of transform files

#Store all of the input files data in their corresponding subject
for i, filepath in enumerate(in_file_list):
    subj_id = filepath.split('/')[-2] #Get the subject id
    unordered_subj_files[subj_id]['in'].append(filepath)
    
for i, filepath in enumerate(ref_file_list):
    subj_id = filepath.split('/')[-2]
    unordered_subj_files[subj_id]['ref'] = filepath
    
for i, filepath in enumerate(tr_file_list):
    subj_id = filepath.split('/')[-2]
    unordered_subj_files[subj_id]['tr'].append(filepath)

    
#Delete the subjects that do not contain at least one of each
subj_to_delete = []
for subj_id in unordered_subj_files:
    if len(unordered_subj_files[subj_id]['in']) == 0:
        if subj_id not in subj_to_delete:
            subj_to_delete.append(subj_id)
    elif unordered_subj_files[subj_id]['ref'] == '':
        if subj_id not in subj_to_delete:
            subj_to_delete.append(subj_id)
    elif len(unordered_subj_files[subj_id]['tr']) == 0:
        if subj_id not in subj_to_delete:
            subj_to_delete.append(subj_id)
for subj_id in subj_to_delete:
    del(unordered_subj_files[subj_id])
    
#TODO: this dictionary is not sorted!
    
""" 
# unordered_subj_files is the dictionary for all subsequent processing
# see below for the variable hierarchy in this dictionary

unordered_subj_files <type 'dict'>
    |
    +-- unordered_subj_files['100307'] <type 'dict'>
    |        |
    |        +-- unordered_subj_files['100307']['in'] <type 'list'>
    |        |        |
    |        |        +-- 100307_input_filepath_1 <type 'str'>
    |        |        +-- 100307_input_filepath_2 <type 'str'>
    |        |        |   ...
    |        |
    |        +-- unordered_subj_files['100307']['ref'] <type 'str'> (reference file)
    |        |
    |        +-- unordered_subj_files['100307']['tr'] <type 'list'>
    |                 |
    |                 +-- 100307_transform_filepath_1 <type 'str'>
    |                 +-- 100307_transform_filepath_2 <type 'str'>
    |                 |   ...?
    |        
    ...        
"""

#Print status lines
print "Total number of subject ID read: %d" % len(IDs)
print "Subjects deleted due to incomplete file paths: %d" % len(subj_to_delete)
print "Remaining number of subjects stored in unordered_subj_files: %d" % len(unordered_subj_files)

#TODO: initiate dataframe that will store all input and ouputs?

Total number of subject ID read: 163
Subjects deleted due to incomplete file paths: 63
Remaining number of subjects stored in unordered_subj_files: 100


In [6]:
# ====================================================================
# Sort subjects, create PARENT output directories
# ====================================================================

# Sort the dictionary by key to create a ordered list of subjects
import collections
ord_subj_files = collections.OrderedDict(sorted(unordered_subj_files.items()))

#Generate output parent directory if it is not present
if not (os.path.isdir(out_parent_dir)):
    os.mkdir(out_parent_dir)
    print "Made output parent directory"
else:
    print "Found output parent directory"
#Generate output jobDoc directory if it is not present
if not (os.path.isdir(jobDoc_dir)):
    os.mkdir(jobDoc_dir)
    print "Made jobDoc directory"
else:
    print "Found jobDoc directory"

Found output parent directory
Found jobDoc directory


In [7]:
# =====================================
# CUSTOMIZATIONS!
# =====================================
# QBatch related
WAITTIME='10:00:00'
CHUNKSIZE='9'
PROCESSOR_PER_JOB='2' #Defalut = 1 

#To submit or not to submit, that is the question
SUBMIT = True

# ====================================================================
# Iterate through subjects to generate job
# ====================================================================

#Open jobDoc and jobList output streams
jobDoc_outStream = open(os.path.join(jobDoc_dir,jobDoc_name),'w')
jobList_outStream = open(os.path.join(jobDoc_dir,jobList_name),'w')

#Write the jobDoc
jobDoc_outStream.write("# Job doc generated on: %s\n" % time.strftime("%Y%d%m-%H%M")) 
jobDoc_outStream.write("# Input files parent folder: %s\n" % in_parent_dir)
jobDoc_outStream.write("# Reference files parent folder %s\n" % ref_parent_dir)
jobDoc_outStream.write("# Transform files parent folder %s\n" % tr_parent_dir)
jobDoc_outStream.write("# Output files parent folder: %s\n#\n" % out_parent_dir)

jobDoc_outStream.write("# antsApplyTransforms specs:\n")
jobDoc_outStream.write("# \tDimensionality: %s\n" % DIMENSIONALITY)
jobDoc_outStream.write("# \tInterpolation: %s\n"% INTERPOLATION)
jobDoc_outStream.write("# Qbatch specs:\n")
jobDoc_outStream.write("# \tWaittime: %s\n" % WAITTIME)
jobDoc_outStream.write("# \tChunksize: %s\n" % CHUNKSIZE)
jobDoc_outStream.write("# \tProcessors per job: %s\n\n" % PROCESSOR_PER_JOB)


jobDoc_outStream.write("qbatch -w %s -c %s --ppj %s %s\nexit\n\n"%(WAITTIME,CHUNKSIZE,PROCESSOR_PER_JOB, os.path.join(jobDoc_dir,jobList_name)))


#Iterate through subjects!
for subj_id in ord_subj_files:
    #Make the output subject directory, if it has not yet been made
    output_subj_dir = os.path.join(out_parent_dir,subj_id)
    if not (os.path.isdir(output_subj_dir)):
        os.mkdir(output_subj_dir)
    #Annotate jobdoc history
    jobDoc_outStream.write("# Subject ID: %s\n" % subj_id)
    
    
    #Iterate through each input file (1 antsApplyTransform per input file)
    for input_file in ord_subj_files[subj_id]['in']:
        #Generate output file path
        output_filename = generate_output_file_name(input_file.split('/')[-1])
        out_filepath = os.path.join(output_subj_dir,output_filename)
        #antsApplyTransform command specifications
        cmd_input=['antsApplyTransforms','-d',DIMENSIONALITY,\
                   '-i',input_file, '-r',ord_subj_files[subj_id]['ref'],\
                   '-o',out_filepath, '-n', INTERPOLATION]
        #Add the transform files
        for tr_file in ord_subj_files[subj_id]['tr']:
            cmd_input += ['-t', tr_file]
        
        #Write out to the jobList file
        jobList_outStream.write(' '.join(cmd_input)+'\n')
                
        
#Close the writing streams
jobDoc_outStream.close()
jobList_outStream.close()

#Automatically submit if indicated
if SUBMIT == True:
    subprocess.call(['bash',os.path.join(jobDoc_dir,jobDoc_name)])

In [ ]:
""" Ant's note to self
- in directory: /data/chamal/projects/anthony/nmf_parcellation/cortical_tractMap/tract2voxel_probability_labels/native_space  
    - the files have 'comp3' instead of 'comp5'. this is likely a mistake in the
        previous step (in Chris' script). will need to fix in bash likely
        to avoid confusion
    - do note that this is fixed in the output file (i added extra
        lines in the output file name generation to make it into comp5
        instead

"""
print "do the above pls"